# Initialisation des librairies

In [1]:
import os
from pyspark.sql.functions import *
from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

# L'objet SPARK Session

In [2]:
spark

In [3]:
spark.sparkContext

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
spark.version

'3.2.0'

# Le chargement des données

In [5]:
os.chdir("/home/razvan/Documents/donnees")

In [6]:
!pwd

/home/razvan/Documents/donnees


In [7]:
!ls -al

total 36
drwxrwxr-x  6 razvan razvan  4096 nov.  29 16:29 .
drwxr-xr-x  5 razvan razvan  4096 nov.  29 16:35 ..
drwxrwxr-x  2 razvan razvan  4096 nov.  29 11:14 .ipynb_checkpoints
drwxrwxr-x  3 razvan razvan 12288 nov.  29 13:40 meteo
drwxr-xr-x 28 razvan razvan  4096 nov.  29 16:29 meteoFrance
-rw-rw-r--  1 razvan razvan  2618 nov.  29 10:10 postesSynop.csv
drwxr-xr-x  2 razvan razvan  4096 nov.  29 16:19 postesSynop.parquet


In [8]:
os.chdir("/home/razvan/Documents/donnees/meteo")

In [9]:
!ls -al

total 1103832
drwxrwxr-x 3 razvan razvan   12288 nov.  29 13:40 .
drwxrwxr-x 6 razvan razvan    4096 nov.  29 16:29 ..
drwxrwxr-x 2 razvan razvan    4096 nov.  29 13:40 .ipynb_checkpoints
-rw-rw-r-- 1 razvan razvan 3253969 mars   4  2015 synop.199601.csv
-rw-rw-r-- 1 razvan razvan 3040309 mars   4  2015 synop.199602.csv
-rw-rw-r-- 1 razvan razvan 3145838 mars   4  2015 synop.199603.csv
-rw-rw-r-- 1 razvan razvan 3190770 mars   4  2015 synop.199604.csv
-rw-rw-r-- 1 razvan razvan 3371785 mars   4  2015 synop.199605.csv
-rw-rw-r-- 1 razvan razvan 3294127 mars   4  2015 synop.199606.csv
-rw-rw-r-- 1 razvan razvan 3348536 mars   4  2015 synop.199607.csv
-rw-rw-r-- 1 razvan razvan 3342032 mars   4  2015 synop.199608.csv
-rw-rw-r-- 1 razvan razvan 3184987 mars   4  2015 synop.199609.csv
-rw-rw-r-- 1 razvan razvan 3247610 mars   4  2015 synop.199610.csv
-rw-rw-r-- 1 razvan razvan 3176893 mars   4  2015 synop.199611.csv
-rw-rw-r-- 1 razvan razvan 3278665 mars   4  2015 synop.199612.csv
-rw-rw-r

# Lecture d'un fichier csv

In [10]:
spark.read.format("csv").option("sep", ";").\
           option("mergeSchema", "true").\
           option("header","true").\
           option("nullValue","mq").\
           load("donnees/postesSynop.csv").\
           filter("ID<8000").\
           toDF("Station","Ville","Latitude","Longitude","Altitude").\
           toPandas().head(5)

,Station,Ville,Latitude,Longitude,Altitude
0,07005,ABBEVILLE,50.136000,1.834000,69
1,07015,LILLE-LESQUIN,50.570000,3.097500,47
2,07020,PTE DE LA HAGUE,49.725167,-1.939833,6
3,07027,CAEN-CARPIQUET,49.180000,-0.456167,67
4,07037,ROUEN-BOOS,49.383000,1.181667,151


In [11]:
donneesStations = spark.read.format("csv").\
                        option("sep", ";").\
                        option("mergeSchema", "true").\
                        option("header","true").\
                        option("nullValue","mq").\
                        load("file:/home/razvan/Documents/donnees/postesSynop.csv").\
                        filter("ID<8000").\
                        toDF("Station","Ville",\
                             "Latitude","Longitude","Altitude")

In [12]:
donneesStations.printSchema()

root
 |-- Station: string (nullable = true)
 |-- Ville: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Altitude: string (nullable = true)



In [13]:
donneesStations.show(5)

+-------+---------------+---------+---------+--------+
|Station|          Ville| Latitude|Longitude|Altitude|
+-------+---------------+---------+---------+--------+
|  07005|      ABBEVILLE|50.136000| 1.834000|      69|
|  07015|  LILLE-LESQUIN|50.570000| 3.097500|      47|
|  07020|PTE DE LA HAGUE|49.725167|-1.939833|       6|
|  07027| CAEN-CARPIQUET|49.180000|-0.456167|      67|
|  07037|     ROUEN-BOOS|49.383000| 1.181667|     151|
+-------+---------------+---------+---------+--------+
only showing top 5 rows



In [14]:
donneesStations.write.mode('overwrite').format('parquet').save('file:/home/razvan/Documents/donnees/postesSynop.parquet')

In [15]:
donnees = spark.read.format('parquet').load('file:/home/razvan/Documents/donnees/postesSynop.parquet')

In [16]:
!ls -al /home/razvan/Documents/donnees

total 36
drwxrwxr-x  6 razvan razvan  4096 nov.  29 16:36 .
drwxr-xr-x  5 razvan razvan  4096 nov.  29 16:35 ..
drwxrwxr-x  2 razvan razvan  4096 nov.  29 11:14 .ipynb_checkpoints
drwxrwxr-x  3 razvan razvan 12288 nov.  29 13:40 meteo
drwxr-xr-x 28 razvan razvan  4096 nov.  29 16:29 meteoFrance
-rw-rw-r--  1 razvan razvan  2618 nov.  29 10:10 postesSynop.csv
drwxr-xr-x  2 razvan razvan  4096 nov.  29 16:36 postesSynop.parquet


In [17]:
donnees.show(5)

+-------+---------------+---------+---------+--------+
|Station|          Ville| Latitude|Longitude|Altitude|
+-------+---------------+---------+---------+--------+
|  07005|      ABBEVILLE|50.136000| 1.834000|      69|
|  07015|  LILLE-LESQUIN|50.570000| 3.097500|      47|
|  07020|PTE DE LA HAGUE|49.725167|-1.939833|       6|
|  07027| CAEN-CARPIQUET|49.180000|-0.456167|      67|
|  07037|     ROUEN-BOOS|49.383000| 1.181667|     151|
+-------+---------------+---------+---------+--------+
only showing top 5 rows



In [18]:
spark.sql("select * from parquet."+
          "`/home/razvan/Documents/donnees/postesSynop.parquet`  "+
          "where Ville = 'ABBEVILLE'").show()

+-------+---------+---------+---------+--------+
|Station|    Ville| Latitude|Longitude|Altitude|
+-------+---------+---------+---------+--------+
|  07005|ABBEVILLE|50.136000| 1.834000|      69|
+-------+---------+---------+---------+--------+



## Chargement des Villes

In [19]:
schema = StructType([
        StructField('Id'           , StringType() , True),
        StructField('ville'        , StringType() , True),
        StructField('latitude'     , FloatType() , True),
        StructField('longitude'    , FloatType() , True),
        StructField('altitude'     , IntegerType() , True)])

villes  = spark.read.format('csv')   \
      .option('sep',';')                \
      .option('mergeSchema', 'true')    \
      .option('header','true')          \
      .schema(schema)                   \
      .load("file:/home/razvan/Documents/donnees/postesSynop.csv")  \
      .cache()

In [20]:
@udf("string")
def formatVille(ville):
    if ville in ['CLERMONT-FD','MONT-DE-MARSAN',
                                   'ST-PIERRE','ST-BARTHELEMY METEO'] :
        return ville.title()
    else :
        if ville.find('-') != -1 :
            return ville[0:ville.find('-')].title()
        else:
            return ville.title()

villesT  = villes.select(
                col('Id').alias('id'),
                formatVille('ville').alias('ville'),
               'latitude',
               'longitude',
               'altitude')

## Chargemet du répértoire meteo

In [21]:
meteoDataFrame  = spark.read.format('csv')\
                       .option('sep',';')\
                       .option('header','true')\
                       .option('nullValue','mq')\
                       .option('inferSchema', 'true')\
                       .load('/home/razvan/Documents/donnees/meteo')\
                       .cache()

In [22]:
meteo = meteoDataFrame.select(
                 col('numer_sta'),
                 to_timestamp(col('date').cast('string'),'yyyyMMddHHmmss'),
                 col('date')[0:4].cast('int') ,
                 col('date')[5:2].cast('int'),
                 col('date')[7:2].cast('int'),
                 col('date')[5:4],
                 round(col('t') - 273.15,2),
                 col('u') / 100 ,
                 col('vv') / 1000 ,
                 col('pres') / 1000,
                 coalesce( col('rr3'),
                           col('rr24')/8,
                           col('rr12')/4,
                           col('rr6')/2,
                           col('rr1')*3  ) )\
             .toDF('id','date','annee','mois','jour','mois_jour','temperature',
                   'humidite','visibilite','pression','precipitations')\
             .cache()

In [23]:
meteo.select('annee','mois','jour','temperature','humidite',
             'visibilite','pression').show(3)

+-----+----+----+-----------+--------+----------+--------+
|annee|mois|jour|temperature|humidite|visibilite|pression|
+-----+----+----+-----------+--------+----------+--------+
| 2019|  12|   1|        3.7|    0.79|      20.0|  100.86|
| 2019|  12|   1|        2.8|    0.87|     12.23|  101.38|
| 2019|  12|   1|        8.7|    0.75|      10.0|  101.39|
+-----+----+----+-----------+--------+----------+--------+
only showing top 3 rows



In [24]:
meteoFance = meteo.where('id < 8000')\
             .join(villesT,'id')\
             .select(initcap(regexp_replace('ville','-',' ')).alias('ville'),
                     'date','annee','mois','jour','temperature','humidite',
                     'visibilite','pression','precipitations')

In [25]:
meteoFance.write\
       .mode('overwrite')\
       .format('parquet')\
       .partitionBy('annee')\
       .option('path', '/home/razvan/Documents/donnees/meteoFrance')\
       .save()

In [26]:
meteoFance.printSchema()

root
 |-- ville: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- annee: integer (nullable = true)
 |-- mois: integer (nullable = true)
 |-- jour: integer (nullable = true)
 |-- temperature: double (nullable = true)
 |-- humidite: double (nullable = true)
 |-- visibilite: double (nullable = true)
 |-- pression: double (nullable = true)
 |-- precipitations: double (nullable = true)



In [27]:
spark.sql("select * from parquet."+
          "`/home/razvan/Documents/donnees/meteoFrance` "+
          "where annee = 2020").show()

+---------------+-------------------+----+----+-----------+--------+----------+--------+--------------+-----+
|          ville|               date|mois|jour|temperature|humidite|visibilite|pression|precipitations|annee|
+---------------+-------------------+----+----+-----------+--------+----------+--------+--------------+-----+
|      Abbeville|2020-03-01 00:00:00|   3|   1|        4.9|     0.8|      20.0|   98.58|           0.0| 2020|
|          Lille|2020-03-01 00:00:00|   3|   1|        5.1|    0.77|      60.0|   98.86|           0.0| 2020|
|Pte De La Hague|2020-03-01 00:00:00|   3|   1|        7.7|    0.81|      10.0|   99.19|           1.8| 2020|
|           Caen|2020-03-01 00:00:00|   3|   1|        6.2|    0.75|     59.95|    98.8|           0.0| 2020|
|          Rouen|2020-03-01 00:00:00|   3|   1|        4.3|    0.86|      60.0|   97.84|           0.0| 2020|
|          Reims|2020-03-01 00:00:00|   3|   1|        6.7|    0.76|      20.0|   98.83|          -0.1| 2020|
|         

In [28]:
meteoFance = spark.read.format('parquet').load('/home/razvan/Documents/donnees/meteoFrance')

In [29]:
meteoFance.printSchema()

root
 |-- ville: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- mois: integer (nullable = true)
 |-- jour: integer (nullable = true)
 |-- temperature: double (nullable = true)
 |-- humidite: double (nullable = true)
 |-- visibilite: double (nullable = true)
 |-- pression: double (nullable = true)
 |-- precipitations: double (nullable = true)
 |-- annee: integer (nullable = true)



In [30]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [31]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+

